## NOTES

#### RESOURCES

In [ ]:
https://brennan.io/2016/03/02/logging-in-with-requests/
https://kazuar.github.io/scraping-tutorial/

In [1]:
import json
import pandas as pd
import csv
import datetime
import collections
import re

from pprint import pprint

from bs4 import BeautifulSoup, NavigableString, Tag
import lxml.html
import requests
from string import capwords
from titlecase import titlecase

pprint('all done')

'all done'


#### COOKIES & LOGIN

In [ ]:
NOT NECCESSARY B/C THE FORUM LIST DOESN'T REQIURE LOGIN TO ACCESS
However, see Prison Talk Scraper Login Notebook for process.

#### REQUEST WEBPAGE AND SAVE TO FILE

In [62]:
s = requests.session()
response = s.get('http://www.prisontalk.com/forums/')

print response.status_code
    
if response.status_code == 200:

    req_forum = BeautifulSoup(response.content, 'lxml')
    
    with open("examples/req_forums.html", "w") as file:
        file.write(str(req_forum))


200


#### LOAD WEBPAGE FROM FILE

In [334]:
html_page = open('examples/req_forums.html', 'r').read()
forum_page = BeautifulSoup(html_page, 'html.parser')

#### REMOVE STRING LITERALS BECAUSE THEY'RE THE WORST

In [335]:
forum_page = str(forum_page)
no_literals = forum_page_str.replace('\n', '').replace('\t', '').replace('\r', '')
forum_page = BeautifulSoup(no_literals, 'html.parser')

#### RETREIVE ALL TOP LEVEL CATEGORY NAMES

In [336]:
# FIND ALL CATEGORY GROUPINGS (E.G. FORUM GROUPINGS)
category_groups = forum_page.find_all(class_="tcat_alt_title")

# REMOVE NON FORUM CATEGORIES (MISC (ARCHIVE), WHAT'S GOING ON (NO FORUMS))
category_groups = category_groups[:-2]

# PARSE CATEGORY GROUP NAMES
category_names = [category_group.a.get_text() for category_group in category_groups]

# CONFIRM THEY ARE OF EQUAL NUMBER
print 'CATEGORY GROUPS:', len(category_groups)
print 'CATEGORY NAMES:', len(category_names)
print ''
for idx, category_name in enumerate(category_names):
    print idx + 1, category_name

CATEGORY GROUPS: 14
CATEGORY NAMES: 14

1 WELCOME TO PTO
2 911
3 BECOMING INVOLVED
4 RESOURCE CENTER
5 MEDICAL & HEALTH
6 FOR FAMILY & FRIENDS
7 LOVING A...
8 CAPITAL PUNISHMENT
9 FOR "OFFENDERS"
10 SHARING EXPERIENCES
11 U.S. REGIONAL FORUMS
12 AROUND THE WORLD
13 PRISON PEN PALS
14 BREAK TIME


#### RETRIEVE ALL TOP LEVEL CATEGORY GROUPS 

NOTE: THE NEXT TWO CAN PROBABLY BE COMBINED

In [357]:
forum_groups = forum_page.find_all(id=re.compile('collapseobj_forumbit'))

# REMOVE NON FORUM CATEGORIES (MISC (ARCHIVE))
forum_groups = forums_groups[:-1]

print 'CATEGORY GROUPINGS:', len(forums_groups)

for idx, forum_grouping in enumerate(forum_groups):
        
    # RETRIEVE ALL SUB TR TAGS BUT REMOVE FIRST ON WHICH IS COL NAMES
    forums = forum_grouping.find_all('tr')[1:]
        
    # PUT IT BACK IN FORUM_GROUPS
    forum_groups[idx] = forums

CATEGORY GROUPINGS: 14


#### REMOVE IRRELEVANT COLUMNS

In [380]:
groups = []

for forum in forum_groups:
    
    for row in forum:
    
        cols = row.find_all('td')
    
        cols = [cols[1], cols[3], cols[4]]

        groups.append(cols)

#### EXTRACT FORUM LIST

In [382]:
forums = []

for group in groups:
    
#     print len(group)
    
    forum_number = get_forum_number(group)
    forum_name = get_forum_name(group)
    num_threads = get_num_threads(group)
    num_posts = get_num_posts(group)
    desc = get_forum_desc(group)
    
    # FUTURE: CHANGE TO DICT FOR DICTWRITER LATER
    forum = [forum_number, forum_name, num_threads, num_posts, desc ]
    
    forums.append(forum)
    
#     print 'NO:', forum_number
#     print 'NAME:', forum_name
#     print 'THREADS:', num_threads
#     print 'POSTS:', num_posts 
#     print 'DESC:', desc
#     print ''

#### WRITE TO CSV (IN THE FUTURE CHANGE TO DICTWRITER)

In [385]:
fieldnames = ['number', 'name', 'threads', 'posts', 'description']

with open('data/forum_list.csv', 'w') as csv_file:
    
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(fieldnames)
    
    for forum in forums:
        writer.writerow(forum)

In [317]:
def get_forum_number(el):
    
    return el[0].a['href'].split('=')[-1]
    
# get_forum_number(groups[0]) # u'33'

In [243]:
def get_forum_name(el):
    
    return el[0].a.get_text()

get_forum_name(groups[0]) # u'PTO News, Information & Events'

u'PTO News, Information & Events'

In [258]:
def get_forum_desc(el):
    
    sub_els = el[0].find_all(class_='smallfont')
    
    return sub_els[0].get_text()

# u"What's new, and what is coming to our web community..."
get_forum_desc(groups[0])

u"What's new, and what is coming to our web community.  Administrative notices and information."

In [248]:
def get_num_threads(el):
    
    return el[1].get_text()

get_num_threads(groups[0]) # u'321'

u'321'

In [249]:
def get_num_posts(el):
    
    return el[2].get_text()

get_num_posts(groups[0]) # u'6,673'

u'6,673'

In [156]:
# THIS COULD BE MORE ELEGANT BY REMOVING ALL SCRIPTS AT THE BEGINNING BUT AH WELL FOR NOW
[script.decompose() for script in forum_categories[0].find_all('script')]

[None]

In [85]:
scripts = forum_categories[0].find_all('script')
scripts[0].decompose()